# 💾 SLM Fine-Tuning: Checkpoint Strategies

---

## Why Do We Need Checkpoints?

Imagine this scenario:

```
Training Progress:
Step 100/500 ████░░░░░░░░░░░░ loss: 2.1
Step 200/500 ████████░░░░░░░░ loss: 1.8
Step 300/500 ████████████░░░░ loss: 1.5
Step 400/500 ████████████████ loss: 1.3

🔴 COLAB DISCONNECTED - GPU SESSION EXPIRED

4 hours of training... gone.
```

**Without checkpoints:** You start from zero. All progress lost.

**With checkpoints:** You resume from step 300. Only 100 steps lost.

---

## The Four Reasons You Need Checkpoints

### 1️⃣ Recovery from Crashes

Training can fail for many reasons:
- Colab session timeout (most common!)
- GPU out of memory
- Network disconnection
- Power outage
- Accidental notebook restart

Checkpoints are your **insurance policy**.

### 2️⃣ Experiment with Different Training Lengths

Not sure if 200 steps or 500 steps is better?

```
checkpoint-100/ → Test this version
checkpoint-200/ → Test this version
checkpoint-300/ → Test this version
```

Compare outputs without retraining!

### 3️⃣ Catch Overfitting

Remember our Part 3 experiment? Loss went UP after step 150.

```
Step 100: loss 1.6  ← Good
Step 150: loss 1.5  ← Best!
Step 200: loss 1.8  ← Overfitting
```

With checkpoints, you can **go back** to checkpoint-150 and use that instead.

### 4️⃣ Share and Deploy Specific Versions

```
checkpoint-200/ → Good for task A
checkpoint-400/ → Good for task B
```

Different checkpoints = different model behaviors.

---

## What's Inside a Checkpoint?

```
./outputs/checkpoint-100/
│
├── adapter_model.safetensors   ← LoRA weights (the A & B matrices)
├── adapter_config.json         ← LoRA settings (r, alpha, targets)
│
├── optimizer.pt                ← Optimizer momentum & state
├── scheduler.pt                ← Learning rate scheduler state
├── trainer_state.json          ← Step count, loss history, etc.
├── rng_state.pth               ← Random seeds (reproducibility)
└── training_args.bin           ← Training configuration
```

| File | For Resuming | For Inference |
|------|:------------:|:-------------:|
| adapter_model.safetensors | ✅ | ✅ |
| adapter_config.json | ✅ | ✅ |
| optimizer.pt | ✅ | ❌ |
| scheduler.pt | ✅ | ❌ |
| trainer_state.json | ✅ | ❌ |

**Key insight:** Inference only needs adapter files. Resuming needs everything.

---

## 1️⃣ Setup

In [1]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.2 MB/s eta 0:00:00


In [2]:
import torch
import os

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU! Change runtime to GPU.")

✅ GPU: Tesla T4


## 2️⃣ Configuration with Checkpoint Strategy

In [3]:
# ============================================================
# CONFIGURATION
# ============================================================

MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_ID = "mlabonne/guanaco-llama2-1k"
OUTPUT_DIR = "./checkpoint_demo"

# Training settings
MAX_SEQ_LEN = 512
BATCH_SIZE = 2
GRAD_ACCUM = 8
MAX_STEPS = 200
LEARNING_RATE = 2e-4

# LoRA settings
LORA_R = 8
LORA_ALPHA = 16

# ============================================================
# CHECKPOINT STRATEGY ⭐
# ============================================================

SAVE_STRATEGY = "steps"    # "steps", "epoch", or "no"
SAVE_STEPS = 50            # Save every 50 steps
SAVE_TOTAL_LIMIT = 4       # Keep only 4 checkpoints (saves disk space)

print("✅ Configuration loaded")
print(f"")
print(f"📁 Checkpoint Strategy:")
print(f"   Save every: {SAVE_STEPS} steps")
print(f"   Keep max: {SAVE_TOTAL_LIMIT} checkpoints")
print(f"   Location: {OUTPUT_DIR}")

✅ Configuration loaded

📁 Checkpoint Strategy:
   Save every: 50 steps
   Keep max: 4 checkpoints
   Location: ./checkpoint_demo


## 3️⃣ Load Dataset, Model, and Apply LoRA

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Dataset
dataset = load_dataset(DATASET_ID, split="train")
print(f"✅ Dataset: {len(dataset)} samples")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
print(f"✅ Model loaded: {MODEL_ID}")

# LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset: 1000 samples


...

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Model loaded: Qwen/Qwen2.5-0.5B-Instruct
trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


## 4️⃣ Training Arguments with Checkpoints

Here's where checkpoint strategy is configured:

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # Training settings
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    max_steps=MAX_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=True,

    # Logging
    logging_steps=10,
    report_to="none",

    # ============================================================
    # CHECKPOINT SETTINGS ⭐
    # ============================================================
    save_strategy=SAVE_STRATEGY,      # When to save: "steps", "epoch", "no"
    save_steps=SAVE_STEPS,            # Save every N steps
    save_total_limit=SAVE_TOTAL_LIMIT,# Max checkpoints to keep

    # Misc
    optim="adamw_torch",
    seed=42,
)

print("✅ Training arguments configured")
print(f"")
print(f"📁 Checkpoints will be saved at steps:")
print(f"   50, 100, 150, 200")
print(f"")
print(f"📁 With save_total_limit={SAVE_TOTAL_LIMIT}, oldest will be deleted")

✅ Training arguments configured

📁 Checkpoints will be saved at steps:
   50, 100, 150, 200

📁 With save_total_limit=4, oldest will be deleted


## 5️⃣ Create Trainer and Train

In [7]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    # max_seq_length=MAX_SEQ_LEN,
)

print("✅ Trainer ready")
print("")
print("🚀 Starting training...")
print("   Watch for checkpoint saves!")

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Trainer ready

🚀 Starting training...
   Watch for checkpoint saves!


In [8]:
trainer.train()

print("\n" + "="*50)
print("✅ Training complete!")
print("="*50)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,1.853000
20,1.875900
30,1.782200
40,1.682800
50,1.737600
60,1.699600
70,1.623700
80,1.692600
90,1.677400
100,1.669400



✅ Training complete!


## 6️⃣ Explore Your Checkpoints

Let's see what was saved:

In [9]:
import os

print("📁 Contents of output directory:")
print("="*50)

for item in sorted(os.listdir(OUTPUT_DIR)):
    item_path = os.path.join(OUTPUT_DIR, item)
    if os.path.isdir(item_path):
        # Calculate folder size
        size = sum(os.path.getsize(os.path.join(item_path, f))
                   for f in os.listdir(item_path)
                   if os.path.isfile(os.path.join(item_path, f)))
        print(f"📂 {item}/ ({size/1024/1024:.1f} MB)")
    else:
        size = os.path.getsize(item_path)
        print(f"📄 {item} ({size/1024:.1f} KB)")

📁 Contents of output directory:
📄 README.md (1.4 KB)
📂 checkpoint-100/ (27.7 MB)
📂 checkpoint-150/ (27.7 MB)
📂 checkpoint-200/ (27.7 MB)
📂 checkpoint-50/ (27.7 MB)


In [10]:
# Look inside a checkpoint
checkpoint_dir = os.path.join(OUTPUT_DIR, "checkpoint-100")

if os.path.exists(checkpoint_dir):
    print(f"📂 Inside {checkpoint_dir}:")
    print("="*50)
    for f in sorted(os.listdir(checkpoint_dir)):
        size = os.path.getsize(os.path.join(checkpoint_dir, f))
        if size > 1024*1024:
            print(f"   {f} ({size/1024/1024:.1f} MB)")
        else:
            print(f"   {f} ({size/1024:.1f} KB)")
else:
    print(f"Checkpoint-100 not found (may have been deleted by save_total_limit)")
    print(f"Available checkpoints: {[d for d in os.listdir(OUTPUT_DIR) if d.startswith('checkpoint')]}")

📂 Inside ./checkpoint_demo/checkpoint-100:
   README.md (5.1 KB)
   adapter_config.json (1.0 KB)
   adapter_model.safetensors (4.1 MB)
   added_tokens.json (0.6 KB)
   chat_template.jinja (2.4 KB)
   merges.txt (1.6 MB)
   optimizer.pt (8.4 MB)
   rng_state.pth (14.3 KB)
   scaler.pt (1.4 KB)
   scheduler.pt (1.4 KB)
   special_tokens_map.json (0.6 KB)
   tokenizer.json (10.9 MB)
   tokenizer_config.json (4.6 KB)
   trainer_state.json (3.4 KB)
   training_args.bin (6.1 KB)
   vocab.json (2.6 MB)


---

# 🔄 Resuming from Checkpoint

## Simulating a Crash

Let's pretend our training crashed at step 100. We'll:
1. Clear the current model from memory
2. Reload everything fresh
3. Resume from checkpoint

In [11]:
# ============================================================
# SIMULATE CRASH: Clear everything
# ============================================================

import gc

# Delete model and trainer
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("💥 SIMULATED CRASH")
print("   Model deleted from memory")
print("   Pretend Colab disconnected here...")

💥 SIMULATED CRASH
   Model deleted from memory
   Pretend Colab disconnected here...


In [12]:
# ============================================================
# RECOVERY: Reload and Resume
# ============================================================

print("🔄 RECOVERING FROM CRASH")
print("="*50)

# Step 1: Reload base model
print("\n1️⃣ Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Step 2: Apply LoRA config (same as before)
print("2️⃣ Applying LoRA configuration...")
model = get_peft_model(model, lora_config)

# Step 3: Create trainer with SAME settings
print("3️⃣ Creating trainer...")

# For resuming, we might want to train MORE steps
resume_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    max_steps=300,  # ⭐ Extended! Was 200, now 300
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=True,
    logging_steps=10,
    report_to="none",
    save_strategy=SAVE_STRATEGY,
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    optim="adamw_torch",
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    args=resume_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    # max_seq_length=MAX_SEQ_LEN,
)

print("\n✅ Ready to resume!")

🔄 RECOVERING FROM CRASH

1️⃣ Loading base model...
2️⃣ Applying LoRA configuration...
3️⃣ Creating trainer...


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
The model is already on multiple devices. Skipping the move to device specified in `args`.



✅ Ready to resume!


In [13]:
# ============================================================
# RESUME TRAINING
# ============================================================

# Find available checkpoints
checkpoints = [d for d in os.listdir(OUTPUT_DIR) if d.startswith('checkpoint')]
print(f"📁 Available checkpoints: {sorted(checkpoints)}")
print("")

# Option A: Resume from LATEST checkpoint automatically
print("🚀 Resuming from latest checkpoint...")
print("="*50)

trainer.train(resume_from_checkpoint=True)  # ⭐ Auto-finds latest

print("\n" + "="*50)
print("✅ Training resumed and completed!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


📁 Available checkpoints: ['checkpoint-100', 'checkpoint-150', 'checkpoint-200', 'checkpoint-50']

🚀 Resuming from latest checkpoint...


Step,Training Loss
210,1.750600
220,1.651000
230,1.641900
240,1.659900
250,1.665700
260,1.644400
270,1.608300
280,1.580500
290,1.675100
300,1.728800



✅ Training resumed and completed!


In [14]:
# ============================================================
# ALTERNATIVE: Resume from SPECIFIC checkpoint
# ============================================================

# If you want to resume from a specific checkpoint:
# trainer.train(resume_from_checkpoint="./checkpoint_demo/checkpoint-100")

print("💡 To resume from a specific checkpoint:")
print("")
print('   trainer.train(resume_from_checkpoint="./checkpoint_demo/checkpoint-100")')

💡 To resume from a specific checkpoint:

   trainer.train(resume_from_checkpoint="./checkpoint_demo/checkpoint-100")


---

# 🔍 Comparing Checkpoints

Now let's load different checkpoints and compare their outputs.

This helps you:
- See how the model evolves during training
- Find the best checkpoint (not always the last one!)
- Understand when overfitting starts

In [15]:
# First, clear memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("🧹 Memory cleared")

🧹 Memory cleared


In [16]:
from peft import PeftModel

# Load base model once
print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)

print("✅ Base model loaded")

Loading base model...
✅ Base model loaded


In [17]:
# ============================================================
# COMPARE OUTPUTS FROM DIFFERENT CHECKPOINTS
# ============================================================

# Test prompt
test_prompt = """### Human: Explain what machine learning is in simple terms.

### Assistant:"""

# Find all checkpoints
checkpoints = sorted([d for d in os.listdir(OUTPUT_DIR) if d.startswith('checkpoint')])
print(f"📁 Found checkpoints: {checkpoints}")
print("\n" + "="*70)

# Compare each checkpoint
for ckpt_name in checkpoints:
    ckpt_path = os.path.join(OUTPUT_DIR, ckpt_name)

    # Load adapter onto base model
    model = PeftModel.from_pretrained(base_model, ckpt_path)
    model.eval()

    # Generate
    inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the assistant's response
    response = response.split("### Assistant:")[-1].strip()

    print(f"\n🔹 {ckpt_name}")
    print("-"*70)
    print(response[:300] + "..." if len(response) > 300 else response)
    print("="*70)

    # Unload adapter to load next one
    del model
    gc.collect()
    torch.cuda.empty_cache()

📁 Found checkpoints: ['checkpoint-150', 'checkpoint-200', 'checkpoint-250', 'checkpoint-300']


🔹 checkpoint-150
----------------------------------------------------------------------
Machine Learning is a subset of artificial intelligence that involves the use of algorithms and statistical models to enable computers to learn from data, identify patterns, and make predictions or decisions. It can be applied to a wide range of tasks such as image recognition, natural language proc...


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(



🔹 checkpoint-200
----------------------------------------------------------------------
Machine Learning is a subset of artificial intelligence that involves the development and application of algorithms and statistical models to enable computers to learn from data, identify patterns, make predictions or decisions, and improve performance over time. It can be used for a wide range of t...

🔹 checkpoint-250
----------------------------------------------------------------------
Machine Learning is a type of artificial intelligence that allows computers to learn and improve their performance on a specific task by analyzing data. It involves using algorithms and statistical models to identify patterns, trends, and relationships within large sets of data, which can then be us...

🔹 checkpoint-300
----------------------------------------------------------------------
Machine learning is a subset of artificial intelligence that involves the development of algorithms and models that enable co

---

# 🔗 Merging Adapter into Base Model

Two ways to use your trained model:

| Method | File Size | Speed | Flexibility |
|--------|-----------|-------|-------------|
| **Keep adapter separate** | Small (~10MB) | Slightly slower | Can swap adapters |
| **Merge into base** | Large (~1GB) | Slightly faster | Single model file |


In [18]:
# ============================================================
# METHOD 1: Load Adapter Separately (Recommended)
# ============================================================

print("Method 1: Loading adapter separately")
print("="*50)

# Choose best checkpoint (let's use the latest)
best_checkpoint = sorted(checkpoints)[-1]
adapter_path = os.path.join(OUTPUT_DIR, best_checkpoint)

print(f"Loading adapter from: {adapter_path}")

# Load
model_with_adapter = PeftModel.from_pretrained(base_model, adapter_path)

print("✅ Model ready for inference!")
print(f"")
print(f"📊 Adapter size: ~{sum(p.numel() for p in model_with_adapter.parameters() if p.requires_grad) * 2 / 1024 / 1024:.1f} MB")

Method 1: Loading adapter separately
Loading adapter from: ./checkpoint_demo/checkpoint-300
✅ Model ready for inference!

📊 Adapter size: ~0.0 MB


In [19]:
# ============================================================
# METHOD 2: Merge Adapter into Base Model
# ============================================================

print("Method 2: Merging adapter into base model")
print("="*50)

# Merge
merged_model = model_with_adapter.merge_and_unload()

print("✅ Adapter merged!")
print(f"")
print(f"📊 Merged model size: ~{sum(p.numel() for p in merged_model.parameters()) * 2 / 1024 / 1024:.1f} MB")

Method 2: Merging adapter into base model
✅ Adapter merged!

📊 Merged model size: ~942.3 MB


In [20]:
# ============================================================
# SAVE MERGED MODEL
# ============================================================

merged_output_dir = "./qwen_finetuned_merged"

print(f"Saving merged model to: {merged_output_dir}")

merged_model.save_pretrained(merged_output_dir)
tokenizer.save_pretrained(merged_output_dir)

print("\n✅ Merged model saved!")
print(f"")
print("📁 Contents:")
for f in os.listdir(merged_output_dir):
    size = os.path.getsize(os.path.join(merged_output_dir, f))
    if size > 1024*1024:
        print(f"   {f} ({size/1024/1024:.1f} MB)")
    else:
        print(f"   {f} ({size/1024:.1f} KB)")

Saving merged model to: ./qwen_finetuned_merged

✅ Merged model saved!

📁 Contents:
   merges.txt (1.6 MB)
   special_tokens_map.json (0.6 KB)
   vocab.json (2.6 MB)
   model.safetensors (942.3 MB)
   generation_config.json (0.2 KB)
   config.json (1.2 KB)
   added_tokens.json (0.6 KB)
   tokenizer.json (10.9 MB)
   chat_template.jinja (2.4 KB)
   tokenizer_config.json (4.6 KB)


![image](Part_4_checkpoints.PNG)

---

# 📝 Quick Reference: Checkpoint Strategies

## Configuration Options

```python
TrainingArguments(
    # When to save
    save_strategy="steps",      # "steps", "epoch", or "no"
    save_steps=100,             # Save every N steps
    
    # How many to keep
    save_total_limit=3,         # Keep only 3 most recent
    
    # Save best model (requires evaluation)
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)
```

## Common Operations

| Task | Code |
|------|------|
| Resume from latest | `trainer.train(resume_from_checkpoint=True)` |
| Resume from specific | `trainer.train(resume_from_checkpoint="./checkpoint-100")` |
| Load adapter | `PeftModel.from_pretrained(base_model, "./checkpoint-100")` |
| Merge adapter | `model.merge_and_unload()` |
| Save merged | `merged_model.save_pretrained("./merged")` |

## Recommended Strategy by Use Case

| Situation | Strategy |
|-----------|----------|
| Quick experiment | `save_strategy="no"`, save at end |
| Long training | `save_steps=100`, `save_total_limit=3` |
| Finding best model | Add evaluation + `load_best_model_at_end=True` |
| Unreliable connection | `save_steps=50` (frequent saves) |

---

## 🎯 Key Takeaways

1. **Checkpoints are insurance** — Always save during long training runs

2. **Use `save_total_limit`** — Don't fill your disk with checkpoints

3. **Resume is easy** — Just pass `resume_from_checkpoint=True`

4. **Compare checkpoints** — Best model isn't always the last one

5. **Two ways to deploy:**
   - Keep adapter separate (small, flexible)
   - Merge into base (standalone model)